In [ ]:
# Atualização das bibliotecas
!pip install --upgrade keras tensorflow

In [ ]:
# Download do dataset de Treinamento
!gdown https://drive.google.com/uc?id=1xQqTSUA-celn_KRB9Qpx77g54KR0WHaH
!unzip Leishmania_Treinamento.zip
!rm Leishmania_Treinamento.zip

In [3]:
# Fixando seeds pra ter resultados mais consistentes
import numpy as np
import tensorflow as tf
import random as python_random

np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [4]:
# Imports
from os import listdir
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
# Inicialização do modelo
classificador = Sequential()

tamanho_imagem = (256, 256)
# Carregamento da arquitetura
transfer = ResNet152V2(include_top = False, input_shape = (tamanho_imagem[0], 
                                                           tamanho_imagem[1], 3))
for camada in transfer.layers:
  camada.trainable = True

classificador.add(transfer)

# Saída do modelo
classificador.add(Flatten())
classificador.add(Dense(1, activation = 'sigmoid'))

classificador.compile(optimizer = Adam(learning_rate = 0.00001),
               loss = 'binary_crossentropy', metrics = ['accuracy'])

batch = 10

# Nenhuma forma de Augmentation
# Rescale é apenas normalização
datagen = ImageDataGenerator(rescale=1./255)


grupo_treinamento = datagen.flow_from_directory('Leishmania_Treinamento/treinamento', 
                                                target_size=tamanho_imagem, 
                                                batch_size=batch, 
                                                class_mode='binary')

grupo_validacao = datagen.flow_from_directory('Leishmania_Treinamento/validacao', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')

classificador.fit(grupo_treinamento, epochs=5, validation_data=grupo_validacao)

234553344/234545216 [==============================] - 10s 0us/step
Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/5
32/32 [==============================] - 16s 491ms/step - loss: 0.6984 - accuracy: 0.6781 - val_loss: 0.5915 - val_accuracy: 0.7000
Epoch 2/5
32/32 [==============================] - 12s 388ms/step - loss: 0.1484 - accuracy: 0.9594 - val_loss: 0.5117 - val_accuracy: 0.7250
Epoch 3/5
32/32 [==============================] - 12s 378ms/step - loss: 0.0617 - accuracy: 0.9906 - val_loss: 0.3983 - val_accuracy: 0.7250
Epoch 4/5
32/32 [==============================] - 12s 391ms/step - loss: 0.0454 - accuracy: 0.9906 - val_loss: 0.3736 - val_accuracy: 0.7625
Epoch 5/5
32/32 [==============================] - 12s 384ms/step - loss: 0.0405 - accuracy: 0.9906 - val_loss: 0.3134 - val_accuracy: 0.8500


In [ ]:
# Download do dataset de Teste
!gdown https://drive.google.com/uc?id=1tTBldPLMQk8jLLDjBaxwbl_FXpYNS0Ux
!unzip Leishmania_Teste.zip
!rm Leishmania_Teste.zip

In [7]:
# Funções

# Preprocessamento das imagens
def carregar_imagem(arquivo):
  img = image.load_img(arquivo, target_size=tamanho_imagem)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste):
  imagens = []
  for pasta in listdir(pasta_teste):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo))
  
  return imagens

In [ ]:
imagens = np.vstack(predicao_teste('Leishmania_Teste/'))
respostas_modelo = classificador.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_Teste/', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
# Marvin pediu especificamente pra gente avaliar os valores de sensibilidade e
# especificidade
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

In [ ]:
# Salvar modelo treinado em um arquivo
classificador.save('modelo_leishmania.h5')